In [191]:
import requests
import time
from tqdm import tqdm

SYSTEM_PROMPT = (
    "Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы."
)
MODEL_NAME = "t-tech/T-lite-it-1.0"

USER_PROMPT_MAIN = '''
Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки.

Запрос пользователя: {query}
Ответ:
'''

USER_PROMPT_POEM = '''
Ты — поэт-ассистент, вдохновляющийся классической русской поэзией.
На основе запроса пользователя сочини **новое** стихотворение в духе русской поэтической традиции.
Можешь использовать метафоры, эпитеты, ритм, рифму — всё, что придаёт тексту поэтичность.
Если знаешь строки из известных произведений — можешь вдохновляться ими, но **не копируй** дословно и не цитируй.
**НЕ ИСПОЛЬЗУЙ** в стихотворении имена и фамилии авторов.

Запрос пользователя: {query}
Стихотворение:
'''

def get_response_rag(query, is_poem):
    if is_poem:
        url = "http://localhost:8001/api/poem_async"
    else:
        url = "http://localhost:8001/api/generate_async"
    res = requests.post(
        url,
        json={"query": query},
        timeout=60
    )
    task_id = res.json()["task_id"]

    for _ in range(10):
        time.sleep(2)
        res = requests.get(f"http://localhost:8001/api/result/{task_id}")
        data = res.json()
        if data.get("status") == "completed":
            return data["result"]["response"]
    else:
        raise('ERROR')


def generate_sync_vllm(query, is_poem):
    if is_poem:
        prompt = USER_PROMPT_POEM.format(query=query)
    else:
        prompt = USER_PROMPT_MAIN.format(query=query)
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1024,
        "temperature": 0.7,
        "top_p": 0.9
    }
    try:
        response = requests.post('http://localhost:8000/v1/chat/completions', json=payload, timeout=60)
        response.raise_for_status()
        result = response.json()
        fin_res = result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"[generate_sync] Ошибка запроса к vLLM: {e}")
        fin_res = ""
    
    if "." in fin_res:
        fin_res = fin_res.rsplit(".", 1)[0] + "."

    return fin_res


In [ ]:
gen_queries = [
    'Напиши стихотворение в стиле Александра Пушкина. Тема - зима в деревне',
    'Придумай стих в стиле Владимира Маяковского на тему советского патриотизма',
    'Напиши стихотворение на тему осеннего дождливого дня как поэт Осип Мандельштам',
    'Сгенерируй стих на тему безответной любви в стиле Ахматовой',
    'Придумай стихотворение на тему одиночества в городе. Пиши в стиле Иосифа Бродского'
]

In [ ]:
gen_response_llm_lst = []
for q in tqdm(gen_queries):
    resp = generate_sync_vllm(q, is_poem=True)
    gen_response_llm_lst.append(resp)

gen_response_llm_lst2 = []
for q in tqdm(gen_queries):
    resp = generate_sync_vllm(q, is_poem=True)
    gen_response_llm_lst2.append(resp)

gen_response_llm_lst3 = []
for q in tqdm(gen_queries):
    resp = generate_sync_vllm(q, is_poem=True)
    gen_response_llm_lst3.append(resp)

In [ ]:
gen_response_rag_lst = []
for q in tqdm(gen_queries):
    resp = get_response_rag(q, is_poem=True)
    gen_response_rag_lst.append(resp)

gen_response_rag_lst2 = []
for q in tqdm(gen_queries):
    resp = get_response_rag(q, is_poem=True)
    gen_response_rag_lst2.append(resp)

gen_response_rag_lst3 = []
for q in tqdm(gen_queries):
    resp = get_response_rag(q, is_poem=True)
    gen_response_rag_lst3.append(resp)

In [238]:
gen_response_rag_lst, gen_response_llm_lst

(['В деревне зимою, где тишь и снег,\nКогда метели белым пухом летят,\nЯ слышу в утреннем часе шаг слуги,\nИ чашка чая, теплая, в руке держу.\n\n"Как холодно?" — спрошу я, "ветер свищет ли?"\n"Пороша есть," — ответит он, "и снег лежит."\nПора в дорогу, скажу я, по седлу сядем,\nИ скачем по полю, где вьюга стонет.\n\nАрапники в руках, собаки за нами,\nИскать следы зверей в этом белом царстве.\nВечером, когда вьюга затихнет,\nСвеча мерцает, и в душе тоска звучит.\n\nКак жажду я просторов бескрайних и вольных!\nСмотрю в окно, где звёзды, словно драгоценности,\nИ вспоминаю, как в детстве, зимним вечером,\nЯ читал книги, мечты мои парили.\n\nНо вновь беру перо, и строки рождаются,\nКак морозный узор на стекле в окне.\nЗима в деревне — это не только труд и покой,\nНо и тайна, что в сердце хранит душа моя.\n\nВот ночь, когда мороз трещит, и в небе бездонном\nСветят звезды, словно жемчужины в тишине.',
  'В светлом храме дней, где времени мечта,\nСветится пламя знамени, как вчера.\nТоварищи, в

In [239]:
gen_response_rag_lst2, gen_response_llm_lst2

(['Зимой, когда стужа в лесу играет  \nИ снежная пелена землю одела,  \nЯ слугу встречу утром с чашкою чая,  \nИ с ним о зиме беседовать начну.  \n\n"Тепло ли в избе? Занялась ли метель?  \nСнегопадом покрыт ли наш край белым?"  \nИ вот уже кони готовы к стремени,  \nИ в поле мы скачем, как ветер, смелым.  \n\nПороши блестящие следы ведут,  \nИ в воздухе пляшут снежинки легкие.  \nА в сердце моем тоска, словно вьюга,  \nНо вспоминаю я дни былые, как драгоценные.  \n\n"Куда ж без тебя, подруга суровая,  \nМне в этих снегах пути не найти.  \nТы ждешь меня в глуши лесной, как маяк,  \nИ вспоминаю, как в детстве мы мечтали".  \n\nВ избе, когда свеча мерцает тусклым светом,  \nИ ветер воет, словно волк, за окном,  \nЯ беру перо и строками создаю,  \nКак ветер песню свою на рассвете.  \n\n"Ты под окном, где снега занесли путь,  \nОжидая, как вехи на жизненном круге,  \nТак и я жду, когда вновь смогу вернуться,  \nИ в сердце твоем найду свой уголок".',
  'Вот так, как ветер, вольный и смелый,

In [240]:
gen_response_rag_lst3, gen_response_llm_lst3

(['Зимой, когда леса в снегу,\nИ мороз играет на окне,\nСело в тишине, словно внуку,\nПодарив тепло своих стен.\n\nВ избе печь потрескивает,\nИ волшебный свет огня\nСветит, как в сказке, в темноте,\nГде снежинки танцуют вдруг.\n\nСедой старик у окна сидит,\nВязанье в руках, и в мыслях даль,\nА рядом внучка с книгой шепчет,\nВ ее глазах – мечты и страх.\n\nСкрипит снег под ногами коней,\nКогда тройка мчится в ночь,\nИ ветер, как песнь, в снежном танце,\nШепчет о зимних днях и ночах.\n\nНо в доме тепло и уютно,\nКак в материнских объятьях.\nЗдесь каждый уголок – как чудо,\nГде зимняя сказка живет.\n\nВ печи пышет огонь, как солнце,\nИ в воздухе аромат хлеба.\nЗдесь, в деревне, зимой так просто\nУвидеть, как светит сердце.\n\n"О, зимняя ночь, ты как поэт,\nТы вдохновляешь на слова,"\n— Прошептал старик, глядя в окно,\nГде луна, как серебряный шар.',
  'Великий день настал, когда сияет свет,\nСвет революции, что в сердце моём живёт.\nКак старший брат, он ведёт нас через бурю,\nСветит всегд

In [ ]:
gen_authors_lst = ['Александр Пушкин', 'Владимир Маяковский', 'Осип Мандельштам', 'Анна Ахматова', 'Иосиф Бродский']
gen_themes_lst = ['Зима в деревне', 'Советский патриотизм', 'Осенний дождливый день', 'Безответная любовь', 'Одиночество в городе']

pd.DataFrame({
    'author': gen_authors_lst,
    'theme': gen_themes_lst,
    'queries': gen_queries,
    'LLM_RAG1': gen_response_rag_lst,
    'LLM1': gen_response_llm_lst,
    'LLM_RAG2': gen_response_rag_lst2,
    'LLM2': gen_response_llm_lst2,
    'LLM_RAG3': gen_response_rag_lst3,
    'LLM3': gen_response_llm_lst3
})#.to_csv('../data/ResponsesGen.csv', index=False)

In [243]:
gen_queries

['Напиши стихотворение в стиле Александра Пушкина. Тема - зима в деревне',
 'Придумай стих в стиле Владимира Маяковского на тему советского патриотизма',
 'Напиши стихотворение на тему осеннего дождливого дня как поэт Осип Мандельштам',
 'Сгенерируй стих на тему безответной любви в стиле Ахматовой',
 'Придумай стихотворение на тему одиночества в городе. Пиши в стиле Иосифа Бродского']